### Importar Bibliotecas ###

In [5]:
#Bibliotecas que realizam a leitura e conversão do PDF para DataFrame
import tabula
from tabula.io import read_pdf
#Bibliotecas para manipulação do DataFrame
import pandas as pd
import numpy as np
#Biblioteca para manipulação de diretórios
import os

In [1]:
#Local onde estão os arquivos PDF da CODESP
path_global = 'C:\\Users\\LDT\\Documents\\GitHub'

### Função de Leitura do PDF CODESP ###

In [3]:
def leitura_codesp(df, page):
    if(page == 15):
        tipo = 'E'
    else:
        tipo = 'I'
    #Preencher Títulos da tabela
    dfTitulos = df[1]
    dfTitulos = dfTitulos.loc[:, ~dfTitulos.columns.str.contains('Unnamed')]
    dfTitulos = dfTitulos.T
    dfTitulos.reset_index(inplace=True)
    dfTitulos = dfTitulos.rename(columns = {'index':'Terminal'})
    dfTitulos = dfTitulos[['Terminal']]
    
    #Preencher Mês do Relatório
    periodo = df[0].iloc[: , -1:]    
    periodo = periodo[df[0].columns[6]]
    periodo = periodo.str.split('-')
    periodo = periodo.tolist()
    periodo = periodo[0][1].strip()
        
    #Leitura dos Resultados
    l = [2,3]
    dfAll = pd.DataFrame(columns={'Período','Operação','Terminal','Tipo','F_E','20_40','Tipo_Valor','Valor'})
    
    for r in l:        
        if (r == 2):
            operacao = 'Longo Curso'
        else:
            operacao = 'Cabotagem'
        
        #Retirando coluna a mais criada somente na tabela cabotagem
        if(r == 3 and page == 15):
            df[r] = df[r].drop(columns=['Unnamed: 1'])
        
        dfResultados = df[r].T
        dfResultados.reset_index(inplace=True)
        dfResultados = dfResultados.rename(columns = {'index':'Quant'})
        dfResultados = dfResultados[2:]

        #Tratamento dos Dados
        dfResultados = dfResultados.rename(columns={dfResultados.columns[0]:'F_20',
                                                    dfResultados.columns[1]:'F_40',
                                                    dfResultados.columns[2]:'F_Soma',
                                                    dfResultados.columns[3]:'E_20',
                                                    dfResultados.columns[4]:'E_40',
                                                    dfResultados.columns[5]:'E_Soma',
                                                    dfResultados.columns[6]:'Total_20',
                                                    dfResultados.columns[7]:'Total_40',
                                                    dfResultados.columns[8]:'Total_Soma'})
        dfResultados = dfResultados.loc[:, ~dfResultados.columns.str.contains('Soma')]
        dfResultados = dfResultados.loc[:, ~dfResultados.columns.str.contains('Total')]
        dfResultados = dfResultados.reset_index()
        dfResultados = dfResultados.loc[:, ~dfResultados.columns.str.contains('index')]
        dfResultados['Tipo'] = ''
        dfResultados.drop(dfResultados.tail(3).index,inplace=True)
        
        
        for column in dfResultados.columns:
                x = 0
                y = 0    
                tipo_valor = ''
                for c in dfResultados[column]:
                    terminal = dfTitulos['Terminal'].iloc[y]
                    if(x%2 == 0):
                        #par
                        tipo_valor = 'Quantidade'
                    else:
                        #impar
                        tipo_valor = 'Peso'
                        y = y + 1

                    dict = {'Período': periodo,'Operação': operacao, 'Terminal':terminal,'Tipo':tipo, 'F_E':column[0:1], '20_40':column[2:],'Tipo_Valor':tipo_valor,'Valor': c}
                    dfAll = dfAll.append(dict, ignore_index = True)       
                    x = x + 1
                    if x == 1:
                        y = 0
                    if y == 5:
                        y = 0
        
        #Filtrando campos de F/E, Quantidade e eliminando 'demais terminais'
        dfAll = dfAll[~dfAll['F_E'].isin(['T'])]
        dfAll = dfAll[dfAll['Tipo_Valor']=='Quantidade']
        dfAll = dfAll[~dfAll['Terminal'].isin(['Demais Pontos'])]

        #Tratando campo valor e corrigindo padrão americano
        dfAll['Valor'] = np.where(dfAll['Valor'].str.contains('Unnamed') == True, 0, dfAll['Valor'])
        dfAll['Valor'] = np.where(dfAll['Valor'].str.contains('-') == True, 0, dfAll['Valor'])
        dfAll['Valor'] = np.where(dfAll['Valor'].str.contains('.') == True, dfAll['Valor'].str.replace(".","").astype('float'), dfAll['Valor'].astype('float'))
        dfAll['Valor'] = dfAll['Valor'].fillna(0)

    return dfAll

### Leitura dos diretórios e chamada para a função Leitura do PDF da CODESP ###

In [7]:
dfCodesp = pd.DataFrame(columns={'Período','Operação','Terminal','Tipo','F_E','20_40','Tipo_Valor','Valor'})
file_path = path_global + "\\2022"
files = os.listdir(file_path)
files_pdf = [f for f in files if f[-3:] == 'pdf']

#Leitura de arquivos
for f in files_pdf:
    path_file = file_path + '\\' + f
    pages = [14,15]
    for page in pages:
        df = read_pdf(path_file, pages=[page], multiple_tables=True,  pandas_options= {'dtype': str})
        data = leitura_codesp(df, page)
        dfCodesp = dfCodesp.append(data)

dfCodesp.to_excel(path_global + '\\codesp_2022_jan_a_abr.xlsx', index=False)

<ipython-input-3-88b0baba6537>:83: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfAll['Valor'] = np.where(dfAll['Valor'].str.contains('.') == True, dfAll['Valor'].str.replace(".","").astype('float'), dfAll['Valor'].astype('float'))
